In [1]:
# importing required libraries
import pandas as pd
import numpy as np
import os
import dask.dataframe as dd


In [2]:
# function for creating a dataframe of security name and Symbol from the meta data file 

def reading_metadata(location):
    #reading metadata information
    meta_df = pd.read_csv(location)

    #keeping_only_reqd_cols from metadata_df
    meta_df = meta_df[['Security Name', 'Symbol']]
    return meta_df


In [3]:
meta_df = reading_metadata("symbols_valid_meta.csv")
meta_df

,Security Name,Symbol
0,"Agilent Technologies, Inc. Common Stock",A
1,Alcoa Corporation Common Stock,AA
2,Perth Mint Physical Gold ETF,AAAU
3,ATA Creativity Global - American Depositary Sh...,AACG
4,AdvisorShares Dorsey Wright ADR ETF,AADR
...,...,...
8044,"Zuora, Inc. Class A Common Stock",ZUO
8045,Zovio Inc. - Common Stock,ZVO
8046,Zymeworks Inc. Common Shares,ZYME
8047,"Zynerba Pharmaceuticals, Inc. - Common Stock",ZYNE


In [4]:
# function for creating a list of tuples with file names and file locations
# csv files of both ETF and stock are saved in a single folder called as directory

def get_file_details(directory):
    nested_dir = os.listdir(directory)
    files = []
    for file in os.listdir(directory+"/"):
        files.append((file, directory+"/"+file))
    return files

In [5]:
a = get_file_details('directory')
a

[('A.csv', 'directory/A.csv'),
 ('AA.csv', 'directory/AA.csv'),
 ('AAAU.csv', 'directory/AAAU.csv'),
 ('AACG.csv', 'directory/AACG.csv'),
 ('AADR.csv', 'directory/AADR.csv'),
 ('AAL.csv', 'directory/AAL.csv'),
 ('AAMC.csv', 'directory/AAMC.csv'),
 ('AAME.csv', 'directory/AAME.csv'),
 ('AAN.csv', 'directory/AAN.csv'),
 ('AAOI.csv', 'directory/AAOI.csv'),
 ('AAON.csv', 'directory/AAON.csv'),
 ('AAP.csv', 'directory/AAP.csv'),
 ('AAPL.csv', 'directory/AAPL.csv'),
 ('AAT.csv', 'directory/AAT.csv'),
 ('AAU.csv', 'directory/AAU.csv'),
 ('AAWW.csv', 'directory/AAWW.csv'),
 ('AAXJ.csv', 'directory/AAXJ.csv'),
 ('AAXN.csv', 'directory/AAXN.csv'),
 ('AB.csv', 'directory/AB.csv'),
 ('ABB.csv', 'directory/ABB.csv'),
 ('ABBV.csv', 'directory/ABBV.csv'),
 ('ABC.csv', 'directory/ABC.csv'),
 ('ABCB.csv', 'directory/ABCB.csv'),
 ('ABEO.csv', 'directory/ABEO.csv'),
 ('ABEQ.csv', 'directory/ABEQ.csv'),
 ('ABEV.csv', 'directory/ABEV.csv'),
 ('ABG.csv', 'directory/ABG.csv'),
 ('ABIO.csv', 'directory/ABIO.c

In [6]:
# function for creating data frames of each csv file with a new column Symbol

def read_and_add(file_info):
    data = dd.read_csv(file_info[1])
    data["Symbol"] = file_info[0][:-4]
    return data

In [7]:
read_and_add(a[0])

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
npartitions=1,,,,,,,,
,object,float64,float64,float64,float64,float64,int64,object
,...,...,...,...,...,...,...,...


In [8]:
# function for concatenating all of the data frames. 

def dask_read(files):
    dfs = [read_and_add(file_info) for file_info in files]
    df = dd.concat(dfs)
    df = df.compute()
    return df

In [9]:
data = dask_read(a)
data

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A
1,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A
2,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A
3,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A
4,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A
...,...,...,...,...,...,...,...,...
4049,2020-03-26,10.230000,11.430000,10.230000,11.100000,11.100000,189500.0,ZYXI
4050,2020-03-27,10.700000,10.980000,10.060000,10.300000,10.300000,145000.0,ZYXI
4051,2020-03-30,10.160000,11.060000,10.160000,10.800000,10.800000,162300.0,ZYXI
4052,2020-03-31,10.680000,11.140000,10.590000,11.070000,11.070000,280400.0,ZYXI


In [10]:
# function for merging meta dataframe and the previously created dataframe with all csv files.

def merge(data, meta_df):
    columns = ["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume", "Symbol", "Security Name"]
    merged_df = meta_df.merge(data, on='Symbol')
    merged_df = merged_df[columns]
    return merged_df


In [11]:
df = merge(data, meta_df)
df

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Security Name
0,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A,"Agilent Technologies, Inc. Common Stock"
1,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A,"Agilent Technologies, Inc. Common Stock"
2,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A,"Agilent Technologies, Inc. Common Stock"
3,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A,"Agilent Technologies, Inc. Common Stock"
4,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A,"Agilent Technologies, Inc. Common Stock"
...,...,...,...,...,...,...,...,...,...
28126845,2020-03-26,10.230000,11.430000,10.230000,11.100000,11.100000,189500.0,ZYXI,"Zynex, Inc. - Common Stock"
28126846,2020-03-27,10.700000,10.980000,10.060000,10.300000,10.300000,145000.0,ZYXI,"Zynex, Inc. - Common Stock"
28126847,2020-03-30,10.160000,11.060000,10.160000,10.800000,10.800000,162300.0,ZYXI,"Zynex, Inc. - Common Stock"
28126848,2020-03-31,10.680000,11.140000,10.590000,11.070000,11.070000,280400.0,ZYXI,"Zynex, Inc. - Common Stock"


In [12]:
# function to convert the resulting dataset into parquet

def write_to_parquet(df, out_loc):
    df.to_parquet(out_loc)

In [13]:
if __name__ == '__main__':
    #location = "symbols_valid_meta.csv"
    #directory = "archive"
    #out_loc = "stage/outpu1.parquet"
    location = input("Enter metadata file full location with the file name: ")
    directory = input("enter the directory where etfs and stocks data is present: ")
    out_loc = input("full location where you want to write the outfile file: ")
    meta_df = reading_metadata(location)
    files = get_file_details(directory)
    df = dask_read(files)
    df = merge(df, meta_df)
    write_to_parquet(df, out_loc)

Enter metadata file full location with the file name:  symbols_valid_meta.csv
enter the directory where etfs and stocks data is present:  directory
full location where you want to write the outfile file:  output1.parquet
